# Hands-On NLP Workshop: *Detecting Sentiment Using Tweets to US Airlines*

Within this hands-on workshop you will analyse tweets from customers of airlines about their performance. These were scraped from Twitter in 2015, and will be categorised in positive, neutral or negative sentiment. 
  
The steps which have been carried out
1. Load The Data 
2. Data Visualization 
3. Text Preprocessing and Cleaning  
4. Handling Imbalance         
5. Model Building  

The code in this notebook has been adapted from the brilliant work done by [Meisam Raz](https://www.kaggle.com/meisamraz/sentiment-analysis-96-acc-eda-text-preprocessing/notebook). 

In [ ]:
# Colab has a load of packages pre-loaded into the environment. Installing the additional ones we require here.
!pip install seldon-deploy-sdk==1.4.1
!pip install alibi-detect==0.6.2
!pip install alibi==0.6.2
!pip install wordcloud
!pip install seaborn
!pip install dill
!pip install imblearn
!pip install nltk

In [ ]:
# Basic Operation
import pandas as pd
import numpy as np

# Text Preprocessing & Cleaning
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import re
from sklearn.model_selection import train_test_split # Split Data 
from imblearn.over_sampling import SMOTE # Handling Imbalanced

# Model Building
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score # Performance Metrics  

# Drift Detection
from alibi_detect.cd import KSDrift
from alibi_detect.utils.saving import save_detector, load_detector

# Explainabilty
from alibi.explainers import AnchorText
import spacy
from alibi.utils.download import spacy_model

# Data Visualization 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from termcolor import cprint
import seaborn as sns
import warnings   

# Serialising artefacts
import joblib
import dill

# Deploy SDK 
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
!wget https://storage.googleapis.com/seldon-koz-data/airline/airline.zip
!unzip -o 'airline.zip'

## Data Exploration

Load the data into memory. 

In [ ]:
df = pd.read_csv('./data/Tweets.csv')
df.head()

Inspecting the columns within the data...

In [ ]:
list(df.columns)

In [ ]:
df.shape

The dataset is 14,460 separate tweets to various US airlines which have been labelled according to their percevied sentiment. There is additional metadata associated with each tweet such as a location of where the tweet was created, the retweet count and the user name of the tweeter. 

Next we can begin to explore and visualise the data!

In [ ]:
cprint("Number of tweets in each category:",'green')
print(df["airline_sentiment"].value_counts())

plt.figure(figsize=(10, 8))
ax = sns.countplot(x='airline_sentiment', data=df, palette='pastel')
ax.set_title(label='Number of tweets in each category', fontsize=20)
plt.show()

Who would have thought? People turn to Twitter when they're annoyed. 

In [ ]:
cprint("Total number of tweets for each airline:",'green')
print(df.groupby('airline')['airline_sentiment'].count())

plt.figure(figsize=(10, 8))
ax = sns.countplot(x='airline', data=df, palette='pastel')
ax.set_title(label='Total number of tweets for each airline', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(24, 10))
sns.countplot(x='negativereason', data=df, palette='hls')
plt.title('Reasons for Negative Tweets', fontsize=20)
plt.show()

So at first glance it appears that customer service issues are the main driver of negatively sentimented tweets!

In [ ]:
NR_Count=df['negativereason'].value_counts()
def NCount(Airline):
    airlineName =df[df['airline']==Airline]
    count= airlineName['negativereason'].value_counts()
    Unique_reason= df['negativereason'].unique()
    Unique_reason=[x for x in Unique_reason if str(x) != 'nan']
    Reason_frame=pd.DataFrame({'Reasons':Unique_reason})
    Reason_frame['count']=Reason_frame['Reasons'].apply(lambda x: count[x])
    return Reason_frame

def plot_reason(airline):
    a= NCount(airline)
    count=a['count']
    Id = range(1,(len(a)+1))
    plt.bar(Id,count, color=['darkviolet','yellow','blue','lime','pink','crimson','gold','cyan','orange','purple'])
    plt.xticks(Id,a['Reasons'],rotation=90)
    plt.title(airline)

airlines= ['US Airways','United','American','Southwest','Delta','Virgin America']
    
plt.figure(2,figsize=(13, 13))
for i in airlines:
    indices= airlines.index(i)
    plt.subplot(2,3,indices+1)
    plt.subplots_adjust(hspace=0.9)
    plot_reason(i)

Breaking that down by airline, we can see that Delta is the only airline whose most frequent issue is late flights. All of the other providers suffer from customer service issues as their leading cause of negative tweets. 

Finally, we can generate word clouds to observe the most common words within each sentiment category. 

In [ ]:
# Split text of Sentiments    
positive = df[df['airline_sentiment'] == 'positive'].text
neutral = df[df['airline_sentiment'] == 'neutral'].text
negative = df[df['airline_sentiment'] == 'negative'].text

In [ ]:
# word cloud of positive sentiments
plt.figure(figsize = (20,20)) 
worldcould_pos = WordCloud(min_font_size=3,
                           max_words=1000, 
                           width=1600,
                           height=680).generate(" ".join(positive))

plt.imshow(worldcould_pos,interpolation='bilinear')
ax.grid(False)

In [ ]:
# word cloud of neutral sentiments
plt.figure(figsize = (20,20)) 
worldcould_neutral = WordCloud(min_font_size=3,  
                               max_words=1000,
                               width=1600,
                               height=680).generate(" ".join(neutral))

plt.imshow(worldcould_neutral,interpolation='bilinear')
ax.grid(False)

In [ ]:
# word cloud of negative sentiments
plt.figure(figsize = (20,20)) 
worldcould_neg = WordCloud(min_font_size=3,
                           max_words=1000, 
                           width=1600, 
                           height=680).generate(" ".join(negative))

plt.imshow(worldcould_neg,interpolation = 'bilinear')
ax.grid(False)

## Data Cleaning

Tweets are inherently messy for ML models to interpret; emojis, punctuation, usernames all need to be removed and prior to being embedded and used for training. 

This section of the notebook tackles the data preparation and cleaning required to wrangle the dataset into a training-ready state. 

Firstly, we prepare the labels by converting the categories to integers. 

In [ ]:
# convert Sentiments to 0,1,2
def convert_sentiment(sentiment):
    if  sentiment == "positive":
        return 2
    elif sentiment == "neutral":
        return 1
    elif sentiment == "negative":
        return 0
    
# Apply convert_sentiment function
df["airline_sentiment"] = df["airline_sentiment"].apply(lambda x : convert_sentiment(x))
df["airline_sentiment"]

Next we then perform a number of data cleaning and standardisation steps. This ensures that there are no quirky characters or usernames fed to our model later down the line which would cause issues. 

In [ ]:
# Remove stop words
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
    return text

# Remove url  
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# Remove punct
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

# Remove html 
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Remove @username
def remove_username(text):
    return re.sub('@[^\s]+','',text)

# Remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Decontraction text
def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text  

# Seperate alphanumeric
def seperate_alphanumeric(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)

def cont_rep_char(text):
    tchr = text.group(0) 
    
    if len(tchr) > 1:
        return tchr[0:2] 

def unique_char(rep, text):
    substitute = re.sub(r'(\w)\1+', rep, text)
    return substitute

def char(text):
    substitute = re.sub(r'[^a-zA-Z]',' ',text)
    return substitute

# Combine negative reason with tweet (if it exists)
df['final_text'] = df['negativereason'].fillna('') + ' ' + df['text'] 

# Apply functions to tweets
df['final_text'] = df['final_text'].apply(lambda x: remove_username(x))
df['final_text'] = df['final_text'].apply(lambda x: remove_url(x))
df['final_text'] = df['final_text'].apply(lambda x: remove_emoji(x))
df['final_text'] = df['final_text'].apply(lambda x: decontraction(x))
df['final_text'] = df['final_text'].apply(lambda x: seperate_alphanumeric(x))
df['final_text'] = df['final_text'].apply(lambda x: unique_char(cont_rep_char,x))
df['final_text'] = df['final_text'].apply(lambda x: char(x))
df['final_text'] = df['final_text'].apply(lambda x: x.lower())
df['final_text'] = df['final_text'].apply(lambda x: remove_stopwords(x))

# End result
df['final_text']

Setting the X and y datasets ready for training. 

In [ ]:
X = df['final_text']
y = df['airline_sentiment']

We then embed the tweets using the TF-IDF vectorizer. This is an industry standard embedding method and provides us with an array for each tweet which can then be used for training a range of ML algorithms. 

In [ ]:
tfidf = TfidfVectorizer()
X_final =  tfidf.fit_transform(X)
X_final

Due to the imbalance in the dataset, with there being 3 times as many negatively sentimented tweets as any other category we use SMOTE to resample the minority classes. 

In [ ]:
smote = SMOTE()
x_sm,y_sm = smote.fit_resample(X_final,y)

Finally, we split the data into training and testing sets. 

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(x_sm , y_sm , test_size=0.2)

## Model Training

Once the data has been prepared we are ready to train our ML model. 

In this case we will use a RandomForestClassifier from Scikit-Learn.

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
rf_prediction =  rf.predict(X_test)

In [ ]:
accuracy_score(rf_prediction,y_test)

Now that we have successfully generated our RandomForestClassifier we can generate a classification report, and visualise a confusion matrix to gain greater insights into our model's performance. 

In [ ]:
cr = classification_report(y_test, rf_prediction)

print("Classification Report:\n----------------------\n", cr)
cm = confusion_matrix(y_test,rf_prediction)

# plot confusion matrix 
plt.figure(figsize=(8,6))
sentiment_classes = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm, cmap=plt.cm.Blues, annot=True, fmt='d', 
            xticklabels=sentiment_classes,
            yticklabels=sentiment_classes)
plt.title('Confusion matrix', fontsize=16)
plt.xlabel('Actual label', fontsize=12)
plt.ylabel('Predicted label', fontsize=12)
plt.show()

## Deploying the Model
As we have seen in the previous sections the tweets are pre-processed using a variety of techniques. In order to account for this we have 2 options for how to account for the pre-processing logic in production:
1. **Custom Model:** Incorporate the pre-processing directly in the `predict` method of a custom model. This provides simplicity when creating the deployment as there is only a single code base to worry about and a single component to be deployed.
2. **Input Transformer:** Make use of a separate container to perform all of the input transformation and then pass the vectors to the model for prediction. The schematic below outlines how this would work. 
```
            ________________________________________
            |            SeldonDeployment          |
            |                                      |
Request -->  Input transformer   -->     Model --> Response
            |  (Pre-processing)          (SKLearn) |
            |______________________________________|
```
The use of an input transformer allows us to separate the pre-processing logic from the prediction logic. This means we can leverage the pre-packaged SKLearn server provided by Seldon to serve our model, and each of the components can be upgraded independently of one another. However, it does introduce additional complexity in the deployment which is generated, and how that then interacts with advanced monitoring components such as outlier and drift detectors. 

This workshop will focus on the generation of a **custom model for this case**, therefore we need to define an `__init__` and `predict` method which shall load and perform inference respectively in our new deployment. 

---------

First we will save our model using joblib- SKLearn's recommended serialisation technology. 

In [ ]:
joblib.dump(rf, 'model.joblib')

We also need to serialise our TF-IDF vectorizer in order to make use of it at runtime. This will ensure that we are using the same embeddings our model relyed upon at training time when inferring against new unseen data. 

In [ ]:
joblib.dump(tfidf, 'tfidf.joblib')

We can now copy the models over to GCS where they can be downloaded from later when we come to deploy. 

!!! Remember to replace your `<YOUR NAME>` with your name. !!!

In [ ]:
!gsutil cp model.joblib gs://tom-seldon-examples/nlp-workshop/<YOUR NAME>/model.joblib
!gsutil cp tfidf.joblib gs://tom-seldon-examples/nlp-workshop/<YOUR NAME>/tfidf.joblib

## Setup

We then define our Seldon custom model. The component parts required to build the custom model are outlined below. Each of the files play a key part in building the eventual Seldon docker container.

---
### TweetSentiment.py
This is the critical file as it contains the logic associated with the deployment wrapped as part of a class by the same name as the Python file. 

A key thing to note about the way this has been structured is that we have focused on making this deployment reusable. The `__init__` method accepts two custom predictor parameters; one for the saved model (`model_path`), and the other for the TF-IDF vectorizer (`tfidf_path`). 

The advantage of this is that it allows us to upgrade the model or vectorizer without having to re-build the container image. Additionally, if the logic was more general it could be used to accept a wider variety of objects for greater reusability. 

```
from joblib import load
import logging
import pandas as pd
import numpy as np
import re
import seldon_core
import os

# For downloading the model and OHE encoder from GCS
from io import BytesIO
from google.cloud import storage

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords", download_dir="./nltk")
nltk.data.path.append("./nltk")

logger = logging.getLogger(__name__)


class TweetSentiment(object):

    def __init__(self, model_path, tfidf_path):
        logger.info(f"Connecting to GCS")
        self.client = storage.Client.create_anonymous_client()
        self.bucket = self.client.bucket('tom-seldon-examples')

        logger.info(f"Model name: {model_path}")
        self.model_path = model_path

        logger.info(f"TF-IDF Name: {tfidf_path}")
        self.tfidf_path = tfidf_path

        logger.info("Loading model file and TF-IDF vectorizer.")
        self.load_deployment_artefacts()
        self.ready = False

    def load_deployment_artefacts(self):
        logger.info("Loading model")
        model_file = BytesIO()
        model_blob = self.bucket.get_blob(f'{self.model_path}')
        model_blob.download_to_file(model_file)
        self.model = load(model_file)

        logger.info("Loading TF-IDF vectorizer")
        tfidf_file = BytesIO()
        tfidf_blob = self.bucket.get_blob(f'{self.tfidf_path}')
        tfidf_blob.download_to_file(tfidf_file)
        self.tfidf = load(tfidf_file)
        
        self.ready = True

    # Remove stop words
    def remove_stopwords(self, text):
        text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
        return text

    ...
    
    def char(self, text):
        substitute = re.sub(r'[^a-zA-Z]',' ',text)
        return substitute

    def predict(self, tweets, names=[], meta={}):
        try:
            if not self.ready:
                self.load_deployment_artefacts()
            else:
                final_text = []

                for text in tweets:
                    # Apply functions to tweets
                    text = self.remove_username(text)
                    text = self.remove_url(text)
                    text = self.remove_emoji(text)
                    text = self.decontraction(text)
                    text = self.seperate_alphanumeric(text)
                    text = self.unique_char(self.cont_rep_char,text)
                    text = self.char(text)
                    text = text.lower()
                    text = self.remove_stopwords(text)
                    final_text.append(text)

                logger.info(f"Final text to be embedded: {final_text}")
                embeddings = self.tfidf.transform(final_text)
                sentiment = self.model.predict(embeddings)
                return sentiment

        except Exception as ex:
            logging.exception(f"Failed during predict: {ex}")
```
---
### .s2i/environment
In order for the Seldon base image to correctly convert your source code to an image it requires certain environment variables. In this case it is only 3 variables. 
* `MODEL_NAME`: The model name matches the name of the Python file and class which is created. 
* `SERVICE_TYPE`: Seldon allows you to create many different components each specialised for a different purpose e.g. `TRANSFORMER` for performing pre or post-processing steps. 
* `PERSISTENCE`: In some cases you would like to save the state of your deployments to Redis e.g. when scaling up multi-armed bandits

This is our environment file:
```
MODEL_NAME=TweetSentiment
SERVICE_TYPE=MODEL
PERSISTENCE=0
```
---
### requirements.txt
List of Python packages which the deployment requires to run.
```
joblib
pandas
numpy
seldon_core
google-cloud-storage
scikit-learn
nltk
```

## Testing Locally
In order to ensure that we have gotten the `TweetClassifier.py` working correctly we can use the `seldon_core` Python package to run our model locally and test the endpoint. 
```
seldon-core-microservice TweetSentiment --service-type MODEL
                                        --parameters='[{ 
                                                        "name": "model_path",
                                                        "value": "nlp-workshop/<YOUR NAME>/model.joblib",
                                                        "type": "STRING"
                                                       }, {
                                                        "name": "tfidf_path",
                                                        "value": "nlp-workshop/<YOUR NAME>/tfidf.joblib",
                                                        "type": "STRING"
                                                       }]'
```
This endpoint can then be tested by posting cURL commands to the local endpoint: 
```
curl -H 'Content-Type: application/json' -d '{"data": {"ndarray": ["@united how can you not put my bag on plane to Seattle. Flight 1212. Waiting  in line to talk to someone about my bag. Status should matter."]}}' http://localhost:9000/api/v1.0/predictions
```

## Building the Image

We can then build the custom model using source 2 image technology. Firstly, installing it locally as per [the documentation](https://github.com/openshift/source-to-image), and then by running this command: 
```
s2i build . seldonio/seldon-core-s2i-python3:1.12.0-dev tweet-sentiment:0.3
```

The built image is then pushed to Dockerhub where it can be pulled ready for deployment. In an enterprise setting, the container registry would be customised to the client's needs. 
```
docker tag tweet-sentiment:0.3 tomfarrand/tweet-sentiment:0.3
docker push tomfarrand/tweet-sentiment:0.3
```

In this case we will use my pre-built container image for speed and simplicity, which we can now deploy using the SDK. 

## Using the SDK
Now that we have our trained model artefact and preprocessor, alongside our custom built container we can now use the Seldon Deploy SDK to deploy our model! 

Initially we setup some authentication: 

In [ ]:
SD_IP = "34.125.69.167"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

Next, we define the parameters which we shall feed to the SDK.

!!! Again, remember to fill in the `YOUR_NAME` parameter !!!

In [ ]:
YOUR_NAME = <YOUR NAME>
MODEL_NAME = "tweet-sentiment"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
CONTAINER_NAME = f"tomfarrand/tweet-sentiment:0.3"

NAMESPACE = "default"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

MODEL_PATH = f"nlp-workshop/{YOUR_NAME}/model.joblib"
TFIDF_PATH = f"nlp-workshop/{YOUR_NAME}/tfidf.joblib"

The deployment specification is then defined. 

In [ ]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "image": CONTAINER_NAME,
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "parameters": [
                        {
                            "name":"model_path",
                            "value":MODEL_PATH,
                            "type":"STRING"
                        },
                        {
                            "name":"tfidf_path",
                            "value":TFIDF_PATH,
                            "type":"STRING"
                        }
                    ],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

Finally, we deploy the model using a few simple API calls. 

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Once our endpoint becomes available we can then test the deployment using the following request: 

```
{"data": {"ndarray": ["@united how can you not put my bag on plane to Seattle. Flight 1212. Waiting  in line to talk to someone about my bag. Status should matter."]}}
```

# Adding an Explainer
Model explainers can be used to generate deeper understanding and insights into why models respond in the manner which they do. 

In this case we will be making use of the Anchors algorithm. The algorithm provides model-agnostic (black box) and human interpretable explanations suitable for classification models applied to images, text and tabular data. The idea behind anchors is to explain the behaviour of complex models with high-precision rules called anchors. These anchors are locally sufficient conditions to ensure a certain prediction with a high degree of confidence.

Initially we provide a function which the `AnchorText` algorithm can call in order to generate a classification. 

In [ ]:
def explainer_predict(x):
    final_x = []
    for text in x:
        text = remove_username(text)
        text = remove_url(text)
        text = remove_emoji(text)
        text = decontraction(text)
        text = seperate_alphanumeric(text)
        text = unique_char(cont_rep_char,text)
        text = char(text)
        text = text.lower()
        text = remove_stopwords(text)
        final_x.append(text)

    embeddings = tfidf.transform(final_x)
    sentiment = rf.predict(embeddings)
    return sentiment

We use spaCy to perform text tokenization, pos-tagging, compute word similarity, etc. The spaCy model can be loaded as follows:

In [ ]:
model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

We then define our explainer itself. The AnchorText explainer acts to perturb a given request in order to generate variations of that request. This can either be done by replacing tokens with an `UNK` token, or by replacing words with similar ones from a corpus. 

In both cases we then observe how the model responds to determine the most important tokens within the request. In this instance we will generate an explainer using the `similarity` strategy. 

In [ ]:
explainer = AnchorText(predictor=explainer_predict, sampling_strategy='similarity', nlp=nlp)

In [ ]:
def convert_prediction(explanation):
    pred = explanation.raw['prediction'][0]
    
    if pred == 0:
        return "neutral"
    elif pred == 1:
        return "negative"
    elif pred == 2:
        return "positive"

def provide_alternatives(prediction):
    classes = ["neutral", "negative", "positive"]
    
    matched_indexes= []
    i = 0
    while i < len(classes):
        if prediction == classes[i]:
            matched_indexes.append(i)
        i += 1
    
    classes.remove(classes[matched_indexes[0]])
    return classes

In [ ]:
explanation = explainer.explain("not sure how to feel about that flight", threshold=0.9)

str_pred = convert_prediction(explanation)

print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)
cprint('\nExamples where anchor applies and model predicts %s:' % str_pred, "green")
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']]))
cprint('\nExamples where anchor applies and model predicts %s:' % provide_alternatives(str_pred), "red")
print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']]))

### Default Text Explainer

Seldon Deploy has a trick up it's sleeve. When deploying Alibi Text explainers we can try to use the default explainer. This is generated on the fly by the Alibi Explain server using the following code: 
```
logging.info("Loading Spacy Language model for %s", spacy_language_model)
spacy_model(model=spacy_language_model)
self.nlp = spacy.load(spacy_language_model)
logging.info("Language model loaded")

self.anchors_text = alibi.explainers.AnchorText(
    predictor=predict_fn, sampling_strategy="unknown", nlp=self.nlp
)
```
Let's see how effective this default explainer is in our case!

We can use this sample request to test our explainer once it is ready: 
```
{"data": {"ndarray": ["not sure how to feel about that flight"]}}
```

## Drift Detection

Although powerful, modern machine learning models can be sensitive. Seemingly subtle changes in a data distribution can destroy the performance of otherwise state-of-the art models, which can be especially problematic when ML models are deployed in production. Typically, ML models are tested on held out data in order to estimate their future performance. Crucially, this assumes that the process underlying the input data `X` and output data `Y` remains constant.

Drift can be classified into the following types:
* **Covariate drift**: Also referred to as input drift, this occurs when the distribution of the input data has shifted `P(X) != Pref(X)`, whilst `P(Y|X) = Pref(Y|X)`. This may result in the model giving unreliable predictions.

* **Prior drift**: Also referred to as label drift, this occurs when the distribution of the outputs has shifted `P(Y) != Pref(Y)`, whilst `P(X|Y) = Pref(X|Y)`. This can affect the model’s decision boundary, as well as the model’s performance metrics.

* **Concept drift**: This occurs when the process generating `Y` from `X` has changed, such that `P(Y|X) != Pref(Y|X)`. It is possible that the model might no longer give a suitable approximation of the true process.

-----------------

In this instance we will train a Kolmgorov-Smirnov drift detector to pick up on covariate drift. The KS Drift detector applies a two-sample KS test to compare the distance between the new probability distribution and the reference distribution. 

This is done on a feature by feature basis and the results are then aggregated using a correction, i.e. Bonferroni, to determine whether drift has occurred overall within the sample. 

We will use the training set as our reference distribution. Creating our drift detector is then as simple as writing a single line of code:

In [ ]:
def preprocess_fn(x_ref):
    final_x = []
    for text in x_ref:
        text = remove_username(text)
        text = remove_url(text)
        text = remove_emoji(text)
        text = decontraction(text)
        text = seperate_alphanumeric(text)
        text = unique_char(cont_rep_char,text)
        text = char(text)
        text = text.lower()
        text = remove_stopwords(text)
        final_x.append(text)

    embeddings = tfidf.transform(final_x).toarray() # Alibi works with np.arrays, be careful about using sparse arrays
    return embeddings

In [ ]:
x_ref = df['text'].to_numpy()
x_ref

In [ ]:
from alibi_detect.cd import KSDrift

dd = KSDrift(x_ref, p_val=.05, preprocess_fn=preprocess_fn)

We can now run predictions against our drift detector, and use it to answer questions about our dataset. For example, do the tweets aimed at Delta differ significantly from the overall tweet corpus?

In [ ]:
delta_tweets = df[df["airline"] == "Delta"]["text"].to_numpy()

preds = dd.predict(delta_tweets)
predspreds = dd.predict(x_ref)
preds

### WARNING: This code is currently not supported with the current Alibi Detect server. 

The use of `preprocess_fn` in this manner is not supported within Alibi v0.6.2, due to the use of `pickle` to serialise objects. However, in later versions of Alibi this is corrected, and therefore should behave correctly.

This will be corrected in the next release of Seldon Deploy in Q1 2022, where we can revisit this example. 

## Conclusion
That draws us to the end of the NLP workshop! 

You have seen how to generate and deploy a custom model for classifying the sentiment of airline tweets. We then created and worked with some of the advanced monitoring and explainability features which are available within the Seldon toolset. 